<a href="https://colab.research.google.com/github/GarapatiVidyaSri/Log-Analysis-Script/blob/main/Welcome_to_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Log Analysis Script**


In [12]:
import re
from collections import Counter, defaultdict
import csv

# Function to parse the log file
def parse_log_file(file_path):
    with open(file_path, 'r') as file:
        logs = file.readlines()
    return logs

# Function to count requests per IP address
def count_requests_per_ip(logs):
    ip_counter = Counter()
    ip_pattern = re.compile(r'^(\S+)')
    for log in logs:
        match = ip_pattern.match(log)
        if match:
            ip_counter[match.group(1)] += 1
    return ip_counter

# Function to identify the most frequently accessed endpoint
def most_accessed_endpoint(logs):
    endpoint_counter = Counter()
    endpoint_pattern = re.compile(r'\"\w+ (\S+) HTTP')
    for log in logs:
        match = endpoint_pattern.search(log)
        if match:
            endpoint_counter[match.group(1)] += 1
    most_common = endpoint_counter.most_common(1)
    return most_common[0] if most_common else None

# Function to detect suspicious activity (failed login attempts)
def detect_suspicious_activity(logs, threshold=5):
    failed_login_counter = Counter()
    failed_login_pattern = re.compile(r'(\S+).+\"POST.+401')
    for log in logs:
        match = failed_login_pattern.search(log)
        if match:
            failed_login_counter[match.group(1)] += 1
    return {ip: count for ip, count in failed_login_counter.items() if count > threshold}

# Function to write results to a CSV file
def write_to_csv(file_name, ip_requests, most_endpoint, suspicious_activity):
    with open(file_name, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)

        # Wrrite Requests per IP
        writer.writerow(['IP Address', 'Request Count'])
        for ip, count in ip_requests.items():
            writer.writerow([ip, count])

        # Write Most Accessed Endpoint
        writer.writerow([])  # Empty line for separation
        writer.writerow(['Endpoint', 'Access Count'])
        if most_endpoint:
            writer.writerow([most_endpoint[0], most_endpoint[1]])

        # Write Suspicious Activity
        writer.writerow([])  # Empty line for separation
        writer.writerow(['IP Address', 'Failed Login Count'])
        for ip, count in suspicious_activity.items():
            writer.writerow([ip, count])

# Main function
def main():
    log_file = 'sample.log'
    logs = parse_log_file(log_file)

    # Analyze logs
    ip_requests = count_requests_per_ip(logs)
    most_endpoint = most_accessed_endpoint(logs)
    suspicious_activity = detect_suspicious_activity(logs)

    # Display results
    print("IP Address\t\tRequest Count")
    for ip, count in ip_requests.most_common():
        print(f"{ip}\t\t{count}")

    print("\nMost Accessed Endpoint:")
    if most_endpoint:
        print(f"Endpoint: {most_endpoint[0]}, Access Count: {most_endpoint[1]}")
    else:
        print("No endpoints found.")

    print("\nSuspicious Activity Detected:")
    if suspicious_activity:
        print("IP Address\t\tFailed Login Attempts")
        for ip, count in suspicious_activity.items():
            print(f"{ip}\t\t{count}")
    else:
        print("No suspicious activity detected.")

    # Write results to CSV
    write_to_csv('log_analysis_results.csv', ip_requests, most_endpoint, suspicious_activity)

if __name__ == '__main__':
    main()


IP Address		Request Count
203.0.113.5		8
198.51.100.23		8
192.168.1.1		7
10.0.0.2		6
192.168.1.100		5

Most Accessed Endpoint:
Endpoint: /login, Access Count: 13

Suspicious Activity Detected:
IP Address		Failed Login Attempts
203.0.113.5		8
